In [ ]:
import os
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import DataLoader, Dataset
from torchvision import transforms, models
from PIL import Image
import pandas as pd
from sklearn.model_selection import train_test_split
import albumentations as A
from albumentations.pytorch import ToTensorV2
import numpy as np
import opendatasets as od  # Библиотека для загрузки датасетов с Kaggle

/usr/local/lib/python3.11/dist-packages/albumentations/__init__.py:24: UserWarning: A new version of Albumentations is available: 2.0.3 (you have 1.4.20). Upgrade using: pip install -U albumentations. To disable automatic update checks, set the environment variable NO_ALBUMENTATIONS_UPDATE to 1.
  check_for_updates()


In [1]:
# !pip install opendatasets

In [ ]:
# Проверка доступности GPU
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(f"Using device: {device}")

Using device: cpu


In [ ]:
# Загрузка датасета с Kaggle
dataset_url = "https://www.kaggle.com/datasets/elmadafri/the-wildfire-dataset"
dataset_path = "/content/the-wildfire-dataset/the_wildfire_dataset_2n_version"

In [ ]:
# Скачиваем датасет, если он еще не скачан
if not os.path.exists(dataset_path):
    od.download(dataset_url)

Please provide your Kaggle credentials to download this dataset. Learn more: http://bit.ly/kaggle-creds
Your Kaggle username:Your Kaggle Key:Dataset URL: https://www.kaggle.com/datasets/elmadafri/the-wildfire-dataset


100%|██████████| 9.94G/9.94G [02:12<00:00, 80.3MB/s]


In [ ]:
# Проверка структуры датасета
print("Содержимое папки датасета:")
print(os.listdir(dataset_path))

# Пути к данным
train_path = os.path.join(dataset_path, "train")
val_path = os.path.join(dataset_path, "val")
test_path = os.path.join(dataset_path, "test")

Содержимое папки датасета:
['test', 'train', 'val']


In [ ]:
# Проверка существования папок
if not os.path.exists(train_path):
    raise FileNotFoundError(f"Папка train не найдена: {train_path}")
if not os.path.exists(val_path):
    raise FileNotFoundError(f"Папка val не найдена: {val_path}")
if not os.path.exists(test_path):
    raise FileNotFoundError(f"Папка test не найдена: {test_path}")


In [ ]:
# Определение аугментаций с использованием albumentations
train_transform = A.Compose([
    A.Resize(224, 224),  # Случайное изменение размера и обрезка
    A.HorizontalFlip(p=0.5),  # Горизонтальное отражение с вероятностью 50%
    A.Rotate(limit=30, p=0.5),  # Поворот на угол до 30 градусов
    A.RandomBrightnessContrast(p=0.2),  # Случайное изменение яркости и контраста
    A.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225]),  # Нормализация
    ToTensorV2(),  # Преобразование в тензор
])

val_transform = A.Compose([
    A.Resize(224, 224),  # Простое изменение размера для валидации
    A.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225]),  # Нормализация
    ToTensorV2(),  # Преобразование в тензор
])

In [ ]:
# Создание пользовательского Dataset
class WildfireDataset(Dataset):
    def __init__(self, folder_path, transform=None):
        self.folder_path = folder_path
        self.transform = transform
        self.classes = [d for d in os.listdir(folder_path) if os.path.isdir(os.path.join(folder_path, d))]  # Список классов (папок)
        self.image_paths = []
        self.labels = []

        # Собираем пути к изображениям и метки
        for class_name in self.classes:
            class_path = os.path.join(folder_path, class_name)
            for img_name in os.listdir(class_path):
                img_path = os.path.join(class_path, img_name)
                # Пропускаем директории и файлы, которые не являются изображениями
                if os.path.isfile(img_path) and img_name.lower().endswith(('.png', '.jpg', '.jpeg')):
                    self.image_paths.append(img_path)
                    self.labels.append(self.classes.index(class_name))
                else:
                    print(f"Ignoring non-image file or directory: {img_path}")

    def __len__(self):
        return len(self.image_paths)

    def __getitem__(self, idx):
        img_path = self.image_paths[idx]
        try:
            image = Image.open(img_path).convert("RGB")
            label = self.labels[idx]

            if self.transform:
                image = self.transform(image=np.array(image))["image"]

            return image, label
        except Exception as e:
            print(f"Error loading image {img_path}: {e}")
            return None, None

In [ ]:
# Создание DataLoader
train_dataset = WildfireDataset(train_path, transform=train_transform)
val_dataset = WildfireDataset(val_path, transform=val_transform)
test_dataset = WildfireDataset(test_path, transform=val_transform)

train_loader = DataLoader(train_dataset, batch_size=32, shuffle=True)
val_loader = DataLoader(val_dataset, batch_size=32, shuffle=False)
test_loader = DataLoader(test_dataset, batch_size=32, shuffle=False)

Ignoring non-image file or directory: /content/the-wildfire-dataset/the_wildfire_dataset_2n_version/val/fire/desktop.ini


In [ ]:
# Загрузка предобученной модели ResNet50
model = models.resnet50(pretrained=True)

# Замена классификатора
num_classes = len(train_dataset.classes)  # Количество классов
model.fc = nn.Linear(model.fc.in_features, num_classes)

# Перемещение модели на устройство
model = model.to(device)

# Оптимизатор и функция потерь
optimizer = optim.Adam(model.parameters(), lr=1e-4)
criterion = nn.CrossEntropyLoss()

/usr/local/lib/python3.11/dist-packages/torchvision/models/_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and may be removed in the future, please use 'weights' instead.
  warnings.warn(
/usr/local/lib/python3.11/dist-packages/torchvision/models/_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=ResNet50_Weights.IMAGENET1K_V1`. You can also use `weights=ResNet50_Weights.DEFAULT` to get the most up-to-date weights.
  warnings.warn(msg)
Downloading: "https://download.pytorch.org/models/resnet50-0676ba61.pth" to /root/.cache/torch/hub/checkpoints/resnet50-0676ba61.pth
100%|██████████| 97.8M/97.8M [00:01<00:00, 54.8MB/s]


In [ ]:
# Обучение модели
num_epochs = 3
for epoch in range(num_epochs):
    model.train()
    running_loss = 0.0
    for images, labels in train_loader:
        images = images.to(device)
        labels = labels.to(device)

        optimizer.zero_grad()
        outputs = model(images)
        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()

        running_loss += loss.item()

    print(f"Epoch {epoch + 1}, Loss: {running_loss / len(train_loader)}")

# Оценка модели на валидационной выборке
model.eval()
correct = 0
total = 0

In [ ]:
with torch.no_grad():
    for images, labels in val_loader:
        if images is None or labels is None:
            continue  # Пропускаем проблемные данные
        images = images.to(device)
        labels = labels.to(device)

        outputs = model(images)
        _, predicted = torch.max(outputs, 1)
        total += labels.size(0)
        correct += (predicted == labels).sum().item()

accuracy = correct / total
print(f"Validation Accuracy: {accuracy * 100:.2f}%")

# Оценка модели на тестовой выборке
correct = 0
total = 0


Validation Accuracy: 91.04%


In [ ]:
with torch.no_grad():
    for images, labels in test_loader:
        if images is None or labels is None:
          continue  # Пропускаем проблемные данные
        images = images.to(device)
        labels = labels.to(device)

        outputs = model(images)
        _, predicted = torch.max(outputs, 1)
        total += labels.size(0)
        correct += (predicted == labels).sum().item()

accuracy = correct / total
print(f"Test Accuracy: {accuracy * 100:.2f}%")

/usr/local/lib/python3.11/dist-packages/PIL/Image.py:3402: DecompressionBombWarning: Image size (101859328 pixels) exceeds limit of 89478485 pixels, could be decompression bomb DOS attack.
  warnings.warn(
/usr/local/lib/python3.11/dist-packages/PIL/Image.py:3402: DecompressionBombWarning: Image size (94487082 pixels) exceeds limit of 89478485 pixels, could be decompression bomb DOS attack.
  warnings.warn(
/usr/local/lib/python3.11/dist-packages/PIL/Image.py:3402: DecompressionBombWarning: Image size (96631920 pixels) exceeds limit of 89478485 pixels, could be decompression bomb DOS attack.
  warnings.warn(


Test Accuracy: 94.39%


In [ ]:
# Сохранение модели
torch.save(model.state_dict(), "resnet50_wildfire.pth")